In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Nlp_Customer_ticket

/content/drive/MyDrive/Nlp_Customer_ticket


In [3]:
pip install dvc[gdrive]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.3/77.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.8/438.8 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.6/118.6 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 461.0/461.0 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.9/374.9 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/

In [4]:
from google.colab import userdata
GITHUB_TOKEN = userdata.get('GIT_NLP_CUSTOM')

In [7]:
#!git remote add origin https://$GITHUB_TOKEN@github.com/Jojo225consulting/Nlp_Customer_ticket.git

In [15]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	modified:   Data.dvc
	new file:   Mlflow_ui.ipynb
	new file:   Nlp_customer_ticket.ipynb
	new file:   Operations.ipynb

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   Operations.ipynb
	modified:   packages_created/__pycache__/preprocessing_data.cpython-311.pyc

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.gitignore
	config.cfg
	mlruns/
	output/



In [14]:
!git add Data.dvc

In [9]:
#!git pull --rebase origin main
#!git pull

In [ ]:
import polars as pl
import pandas as pd
import numpy as np

In [ ]:

import sys
sys.path.append("/content/drive/MyDrive/Nlp_Customer_ticket/packages_created")

In [ ]:
import preprocessing_data as ppd

In [ ]:
import spacy
from spacy.tokens import DocBin

In [ ]:
df=ppd.make_clean("/content/drive/MyDrive/Nlp_Customer_ticket/Data/Raw/aa_dataset-tickets-multi-lang-5-2-50-version.csv")
df_train = df.filter(pl.col("language") == "en")

df_test = ppd.make_clean("/content/drive/MyDrive/Nlp_Customer_ticket/Data/Raw/dataset-tickets-multi-lang3-4k.csv")
df_test=df_test.filter(pl.col("language")=="en")

No difference between after cleaning
Some elements have been dropped during cleaning


In [ ]:
Unique_queue = {'Technical Support', 'Product Support','Customer Service', 'IT Support'}
df_train=pd.DataFrame(df_train, columns=df_train.columns)
df_test=pd.DataFrame(df_test, columns=df_test.columns)
df_train["queue_treated"]=df_train['queue'].apply(lambda x: x if x in Unique_queue else 'Others')
df_test["queue_treated"]=df_test['queue'].apply(lambda x: x if x in Unique_queue else 'Others')

In [ ]:
df_train["queue_treated"].value_counts()*100/sum(df_train["queue_treated"].value_counts())

,count
queue_treated,
Technical Support,28.993757
Others,25.560044
Product Support,18.808912
Customer Service,14.750888
IT Support,11.886400


In [ ]:
df_test["queue_treated"].value_counts()*100/sum(df_test["queue_treated"].value_counts())

,count
queue_treated,
Technical Support,33.669065
Others,22.158273
Product Support,16.834532
Customer Service,15.323741
IT Support,12.014388


In [ ]:
labels = list(df_train["queue_treated"].unique())

TRAIN_DATA = []
for _, row in df_train.iterrows():
    cats = {label: 1.0 if label == row["queue_treated"] else 0.0 for label in labels}
    TRAIN_DATA.append((row["body"], {"cats": cats}))


In [ ]:
labels = list(df_test["queue_treated"].unique())

TEST_DATA = []
for _, row in df_test.iterrows():
    cats = {label: 1.0 if label == row["queue_treated"] else 0.0 for label in labels}
    TEST_DATA.append((row["body"], {"cats": cats}))

In [ ]:
def convert_to_spacy(data, output_file):
    nlp = spacy.blank("en")
    db = DocBin()
    for text, annotations in data:
        doc = nlp.make_doc(text)
        doc.cats = annotations["cats"]
        db.add(doc)
    db.to_disk(output_file)

convert_to_spacy(TRAIN_DATA[:10000], "train_queue_gathering.spacy")
convert_to_spacy(TRAIN_DATA[10000:], "valid_queue_gathering.spacy")
convert_to_spacy(TEST_DATA, "test_queue_gathering.spacy")

In [ ]:
!cp -r /content/drive/MyDrive/Nlp_Customer_ticket/test_queue_gathering.spacy /content/drive/MyDrive/Nlp_Customer_ticket/Data/Spacy_format
!cp -r /content/drive/MyDrive/Nlp_Customer_ticket/train_queue_gathering.spacy /content/drive/MyDrive/Nlp_Customer_ticket/Data/Spacy_format
!cp -r /content/drive/MyDrive/Nlp_Customer_ticket/valid_queue_gathering.spacy /content/drive/MyDrive/Nlp_Customer_ticket/Data/Spacy_format

In [ ]:
!ls /content/drive/MyDrive/Nlp_Customer_ticket/Data/Spacy_format

test_queue_gathering.spacy   train.spacy
test.spacy		     valid_queue_gathering.spacy
train_queue_gathering.spacy  valid.spacy
